In [63]:
from bs4 import BeautifulSoup
import urllib2
import requests
import pandas as pd

In [64]:
source=requests.get('http://myneta.info/ls2014/index.php?action=summary&subAction=candidates_analyzed&sort=candidate#summary')

In [65]:
tree= BeautifulSoup(source.text,"html.parser")

In [66]:
len(tree)

22

In [67]:
tree.title

<title>Total candidates analyzed by NEW:Loksabha 2014 Election</title>

In [68]:
all_links = tree.find_all('a')

In [69]:
table_pol = tree.findAll('table')[2]
rows = table_pol.findAll("tr")[2:]

In [70]:
# We build one dictionary per candidate #

def c_link(r):
    return r.find("a").get("href")
def name(r):
    return r.find("a").get_text()
def cols(r):
    return r.findAll("td")
def assets(r):
    col = cols(r)
    ass = col[6].get_text().split("~")[0].encode('ascii', 'ignore').replace("Rs","").replace(",","")
    return ass
def liab(r):
    col = cols(r)
    liabilities = int(col[7].get_text().split("~")[0].encode('ascii', 'ignore').replace("Rs","").replace(",",""))
    return liabilities

info_candidate = lambda r: [int(cols(r)[0].get_text()), r.find("a").get("href"),r.find("a").get_text(),
                            cols(r)[2].get_text(),cols(r)[3].get_text(),cols(r)[5].get_text(),
                            cols(r)[4].get_text(),assets(r), liab(r)]

title = ['id','url','name','district','party','education','nr_crime','assets','liabilities']
dict_candidates = [dict(zip(title,info_candidate(r))) for r in rows]

In [71]:
len(dict_candidates)
dict_candidates[0:5]

[{'assets': '154566136 ',
  'district': u'NAWADA',
  'education': u'Post Graduate',
  'id': 1,
  'liabilities': 2604969,
  'name': u'Kaushal Yadav',
  'nr_crime': u'8',
  'party': u'JD(U)',
  'url': u'candidate.php?candidate_id=148'},
 {'assets': '2191523 ',
  'district': u'AZAMGARH',
  'education': u'Others',
  'id': 2,
  'liabilities': 0,
  'name': u'M. Aamir Rashadi',
  'nr_crime': u'1',
  'party': u'Rashtriya Ulama Council',
  'url': u'candidate.php?candidate_id=9496'},
 {'assets': '306023 ',
  'district': u'MAHARAJGANJ',
  'education': u'Graduate Professional',
  'id': 3,
  'liabilities': 0,
  'name': u'Rakesh Kumar Giri',
  'nr_crime': u'0',
  'party': u'IND',
  'url': u'candidate.php?candidate_id=9706'},
 {'assets': '3630000 ',
  'district': u'CHENNAI SOUTH',
  'education': u'8th Pass',
  'id': 4,
  'liabilities': 850000,
  'name': u'(Kuppal)G.Devadoss',
  'nr_crime': u'0',
  'party': u'IND',
  'url': u'candidate.php?candidate_id=6912'},
 {'assets': '44357368 ',
  'district': u'

In [72]:
# Now we create a really big dictionary which stores url and page for each candidate
# Work in progress...
def url_aggregator(url):
    full_url = "http://myneta.info/ls2014/"+url
    page = requests.get(full_url)
    url_page = [url, page]
    return url_page

#titles = ['url','id_page']
#dict_allpages = dict(zip(titles,url_aggregator(url)) for url in dict_candidate['url'])

In [73]:
# Now each candidate has an url - so later we  will parse each of their pages and scrape them#
#But first let's look how the page looks like for a few individual candidates
short_url = dict_candidates[1555]['url']
full_url = "http://myneta.info/ls2014/"+short_url
print full_url
page_candidate = requests.get(full_url)
c_soup = BeautifulSoup(page_candidate.text,"html.parser")

http://myneta.info/ls2014/candidate.php?candidate_id=2885


In [74]:
#### Personal info #########

personal = c_soup.findAll(attrs={"class": "grid_3 alpha"})[0]
full_name = personal.find("h2").get_text()
district1 = personal.find("h5").get_text()
grid2 = personal.findAll(attrs={"class":"grid_2 alpha"})
party_full = grid2[0].get_text().split(":")[1]
age = int(grid2[2].get_text().split(":")[1])
address = grid2[3].get_text().split(":")[1].split("\n")[1] # Careful this one changes
self_profession = personal.find("p").get_text().split('\n')[0].split(":")[1]
spouse_profession = personal.find("p").get_text().split('\n')[1].split(":")[1]
print full_name, "\n works as a:", self_profession, ". \n Profession of spouse:", spouse_profession, ". \n Address: ",
print address, ", in district: ", district1, "for party:", party_full, "The dude is", age, "years old"

########## Assets  & Criminality ###########
# Ok let's get serious and find his assets and criminal activity
all_tables = c_soup.findAll("table")
print "\n \n \n" , len(all_tables)
table_inc = all_tables[0]
table_crim = all_tables[1]
table_assets1 = all_tables[2]
table_assets2 = all_tables[3]
table_liab = all_tables[4]
#print table_inc
# That does not really work: some have a table for criminal activities, others not
# So we need more work to know exactly how to have the right table correspond to the right thing


DASHRATH GAGRAI  
 works as a: Agriculture . 
 Profession of spouse: Agriculture & Para Teacher . 
 Address:              R/O Vill-Losodiki,Po jojo Kudma,Panchayat Telahidih,Parkhand Kharsawa Thana Kharsawa,Distt-Saraykela Kharsawa,Pin 833102 , in district:  
        SINGHBHUM  (JHARKHAND)
         for party: JVM(P)
  The dude is 38 years old

 
 
9
<table class="table1" width="100%"><tr valign="top"><td>
<h3 class="title blue">Cases where charges framed</h3>
<table border="1" cellpadding="1" width="80%">
<tr style="font-weight:bold"><td>Serial No.</td><td>IPC Sections Applicable</td><td>Other Details / Other Acts / Sections Applicable</td></tr>
<tr><td> 1</td><td>426, 427 </td><td> P.s.Chakardharpur Case no 152/10 ,08.02.09 (G.R.Case no.327/2009)10.05.2010 West Sighbhoom Chaibasa Jharkhand 3PDP Act Sub Divisinol Judicial Magistrate Podahat Chaibasa 19.02.03  Case pending For Witness  </td></tr>
</table>
<h3 class="title blue">Cases where Cognizance taken</h3>
<table border="1" cellpad